In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [12]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

KeyError: "['link'] not found in axis"

In [11]:
df.head(10)

,artist,song,text
0,Happy Mondays,Bobs Yer Uncle,What do you want to hear when we're making lov...
1,David Bowie,London Boys,Bow Bells strike another night \r\nYour eyes ...
2,Omd,The More I See You,The more I see you \r\nThe more I want you \...
3,Billy Joel,She's Got A Way,She's got a way about her \r\nI don't know wh...
4,Mariah Carey,Little Drummer Boy,"Come, they told me \r\nPa, rum, pa, pum, pum ..."
5,Alan Parsons Project,Day After Day,Lead vocal: jack harris \r\n \r\nGaze at the...
6,Andy Williams,Born Free,"Born free, as free as the wind blows \r\nAs f..."
7,Violent Femmes,Breakin' Up,Now every once in a while \r\nThere's a crack...
8,George Strait,Nobody In His Right Mind Would've Left Her,I burn with desire \r\neach time my heart fan...
9,John McDermott,The Dutchman,The Dutchman's not the kind of man \r\nTo kee...


In [13]:
df['text'][0]

"What do you want to hear when we're making love  \r\nWhat do you want to hear when we're making love  \r\nCan I take you from behind and hold you in my arms  \r\nWhat do you want to hear when we're making love  \r\n  \r\nCan I take you from behind and feel you in my heart  \r\nWhat do we need to re-live to bring us close  \r\nWhy don't you do it to me  \r\nWhy don't you do those things to me  \r\n  \r\nWhy don't you do it to me  \r\nWhy don't you do those things to me  \r\nFour fall in a bed, three giving head, one getting wet  \r\nFour fall in a bed, three giving head, one getting wet  \r\n  \r\nWhat do you want to hear when we're making love  \r\nCan I hold you from behind and tell you that it's me  \r\nWhy don't you do it to me  \r\nWhy don't you do those things to me  \r\n  \r\nWhy don't you do it to me  \r\nWhy don't you do those things to me  \r\nThe love drug is a bug that cuts us both up  \r\nWhy don't you do those things to me  \r\n  \r\nWhy don't you do it of me, why don't y

In [14]:
df.shape

(5000, 3)

In [ ]:
#Text Cleaning and preprocessing

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [17]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [20]:
similarity[0]

array([1.        , 0.03154022, 0.09044986, ..., 0.00947776, 0.01722638,
       0.0254167 ])

In [21]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [25]:

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))